In [1]:
import cython
%load_ext cython

In [20]:
%%cython
import numpy as np
cimport numpy as np

cdef magnitude(np.ndarray p, np.ndarray q):
    cdef double dx = q[0] - p[0]
    cdef double dy = q[1] - p[1]
    cdef double res = np.sqrt(dx*dx + dy*dy)
    return res

def dist_pt_seg(np.ndarray p, np.ndarray u, np.ndarray v):
    cdef double seglen = magnitude(u, v)
    if seglen < 1e-8:
        raise Exception('segment is co-linear')
    t =  (p[0] - u[0]) * (v[0] - u[0])
    t += (p[1] - u[1]) * (v[1] - u[1])
    t /= seglen**2
    print t
    if t < 0.0:
        return magnitude(p, u)
    elif t > 1.0:
        return magnitude(p, v)
    else:
        return magnitude(p, u + t * (v-u)) 

In [23]:
p = np.array([-2.0, 1.0])
u = np.array([-1.0, 0.0])
v = np.array([ 1.0, 0.0])
dist_pt_seg(p, u, v)

-0.5


1.4142135623730951

Original Version

    #include <cmath>
    #define EPS 1e-8

    struct Point2
    {
        float X;
        float Y;
    };

    double magnitude( Point2 *p, Point2 *q )
    {
        Point2 delta;

        delta.X = q->X - p->X;
        delta.Y = q->Y - p->Y;

        return sqrt( delta.X * delta.X + delta.Y * delta.Y );
    }

    int dist_pt_seg( Point2 *p, Point2 *u, Point2 *v, double *result)
    {
        Point2 intersection;
        double seglen;
        double t;

        seglen = magnitude( u, v );
        if( seglen < EPS ) {
            return 0;
        }

        t = ( ( ( p->X - u->X ) * ( v->X - u->X ) ) +
              ( ( p->Y - u->Y ) * ( v->Y - u->Y ) ) ) /
              (seglen * seglen);

        if( t < 0.0f ) {
           *result = magnitude( p, u );
        }
        else if (t > 1.0f ) {
            *result = magnitude( p, v );
        }
        else {
            intersection.X = p->X + t * ( v->X - u->X );
            intersection.Y = p->Y + t * ( v->Y - u->Y );
            *result = magnitude( p, &intersection );
        }
        return 1;
    }